In [1]:
import pandas as pd
import pyomo.environ as pe
import preprocessing
from optimisation import *
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import model
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import swifter

In [2]:
train = pd.read_csv("data/train.csv")

In [3]:
test = pd.read_csv("data/test.csv")

In [4]:
train_upd, test_upd = preprocessing.preprocessing_all(train, test)

In [5]:
test_out, y_prob_model, y_model = model.run_model_and_distrs(train_upd, test_upd, percent_val=0.2, qunity_starts=6, quantity_points_out=100, round_digits=6)


e: 0.0258716	valid_0's l2: 0.00066934
[420]	valid_0's rmse: 0.0258607	valid_0's l2: 0.000668776
[421]	valid_0's rmse: 0.025853	valid_0's l2: 0.000668377
[422]	valid_0's rmse: 0.0258482	valid_0's l2: 0.000668129
[423]	valid_0's rmse: 0.0258404	valid_0's l2: 0.000667725
[424]	valid_0's rmse: 0.0258374	valid_0's l2: 0.000667574
[425]	valid_0's rmse: 0.0258313	valid_0's l2: 0.000667258
[426]	valid_0's rmse: 0.0258229	valid_0's l2: 0.000666823
[427]	valid_0's rmse: 0.0258143	valid_0's l2: 0.00066638
[428]	valid_0's rmse: 0.0258116	valid_0's l2: 0.000666239
[429]	valid_0's rmse: 0.0258063	valid_0's l2: 0.000665967
[430]	valid_0's rmse: 0.0258052	valid_0's l2: 0.000665908
[431]	valid_0's rmse: 0.0257973	valid_0's l2: 0.000665501
[432]	valid_0's rmse: 0.0257892	valid_0's l2: 0.000665082
[433]	valid_0's rmse: 0.0257842	valid_0's l2: 0.000664825
[434]	valid_0's rmse: 0.0257778	valid_0's l2: 0.000664497
[435]	valid_0's rmse: 0.0257688	valid_0's l2: 0.000664033
[436]	valid_0's rmse: 0.025766	valid

In [49]:
y_prob_model = np.round(y_prob_model/y_prob_model.sum(axis=1)[:, None], 6)

In [51]:
pd.DataFrame(y_prob_model).to_excel('tmp.xlsx')

In [30]:
quantity, res = np.histogram(residuals, bins=100)
quantity = quantity/sum(quantity)

In [2]:
opt = galaxy_optim(train)
opt.solve()
results = opt.prepare_output_file()
opt.check_optim_results()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1823.96835259911
  Upper bound: 1823.96835259911
  Number of objectives: 1
  Number of constraints: 1783
  Number of variables: 891
  Number of nonzeros: 3561
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.028393983840942383
# ----------------------------------------------------------
#   Solution Information
# ------